## MNIST - Multi-Layer-Perceptron - Tensorflow

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('MNIST_data/',one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [35]:
# No. of training cycles and batches to feed the net
epochs = 5000
batch_size = 256

#Input and Outer layer size to create the multi-layer
input_layer = mnist.train.images.shape[1]
output_classes = mnist.train.labels.shape[1]

#No. of hidden layer and nodes in them
layers = [512,256,128]

#Placeholders for feeding the batches 
x = tf.placeholder("float",[None,input_layer])
y = tf.placeholder("float",[None,output_classes])


#### The function below will generate all the layers for the net based on  the input, output and hidden layers.

In [36]:
def generate_model(n_input = input_layer,n_classes = output_classes,layers =layers):
    
    dimension_1 = [n_input] + layers
    dimension_2 = layers + [n_classes]
    
    names = ['h' + str(i+1) for i in range(len(layers))] + ['out']
    
    weights_var = [tf.Variable(tf.random_normal([i,j])) for i,j in zip(dimension_1,dimension_2)]
    biases_var  = [tf.Variable(tf.random_normal([i])) for i in (dimension_2)]
    
    weights_1 = dict(zip(names,weights_var))
    biases_1 = dict(zip(names,biases_var))
    #print(weights_1)
    #print(biases_1)
    model = x
    
    for key in sorted(weights_1):
        #print (model)
        model = tf.add(tf.matmul(model,weights_1[key]),biases_1[key])
        if key != 'out':
            model = tf.nn.relu(model)
        #print(model)    
    return(model)

In [37]:
#Generate the model (multi-layered net)
pred = generate_model()

#Optimization routine
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred,y))
optimizer = tf.train.RMSPropOptimizer(0.001,0.9).minimize(cost)

#Estimation of the accuracy
correct_pred = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

### Running the net

In [38]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(epochs):
        
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        sess.run(optimizer,feed_dict = {x:batch_x,y:batch_y})

        if epoch % 1000 == 0 :
            loss,acc = sess.run([cost,accuracy],feed_dict={x:batch_x,y:batch_y})
            print ("Cost = {0}, Training Accuracy = {1}".format(loss,acc))
            
    test_acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
    print ("TEST ACCURACY = {}".format(test_acc))   

Cost = 14007.3212890625, Training Accuracy = 0.1171875
Cost = 80.15953063964844, Training Accuracy = 0.98046875
Cost = 1.3012657165527344, Training Accuracy = 0.99609375
Cost = 3.86496950000037e-08, Training Accuracy = 1.0
Cost = 1.2406177520751953, Training Accuracy = 0.99609375
TEST ACCURACY = 0.9463000297546387
